# Sketchpad GIS

This notebook was used to gain knowledge on how to get data through PDOK's API (especially how to get all data from within a certain bounding box rather than the entirety of The Netherlands), and how to wrangle with polygons. It was an experimental playground, which has been heavily edited with parts being added, removed, and edited as the situation called for it. As such, this sketchpad does not follow proper code conventions and should not be seen as representative for the final product.

All this code is not necessary and is not used within the final pipeline. In the end, a direct download (wget) was used as the dataset was hosted under a single 10GB download link through Nationaal GeoRegister (NGR). 

- PDOK dataset: https://api.pdok.nl/brt/top10nl/ogc/v1/
- Same dataset on NGR: https://nationaalgeoregister.nl/geonetwork/srv/dut/catalog.search#/metadata/29d5310f-dd0d-45ba-abad-b4ffc6b8785f
- Download link dataset: https://service.pdok.nl/brt/topnl/atom/downloads/top10nl_Compleet-2024.gpkg

As this consists largely of code not used in the final pipeline, it lacks proper documentation, and is largely kept for archival purposes.

## Grabbing data from PDOK

In [ ]:
import requests
import json
import os


API_BASE_URL = 'https://api.pdok.nl/brt/top10nl/ogc/v1'
TARGET_COLLECTIONS = ['waterdeel_vlak', 'terrein_vlak']
LIMIT = 10000
OUTPUT_DIR = 'all_boundaries_geojson'

# Bounding Box for Utrecht province, OGC-standard for CRS84 (longitude, latitude).
BBOX_UTRECHT_CRS84 = '4.75,51.90,5.57,52.39'

# De URI die het coördinatensysteem voor de bounding box specificeert.
# Dit is de standaard-identifier voor WGS84 in OGC API's.
BBOX_CRS_URI = 'http://www.opengis.net/def/crs/OGC/1.3/CRS84'


def download_features_for_collection(collection_id):
    print(f"\n--- Starten met downloaden van collectie: {collection_id} ---")
    
    features_url = f"{API_BASE_URL}/collections/{collection_id}/items"
    
    params = {
        'f': 'json',
        'bbox': BBOX_UTRECHT_CRS84,
        'bbox-crs': BBOX_CRS_URI,
        'limit': LIMIT
    }
    
    all_features = []
    page_num = 1
    
    while features_url:
        try:
            request_url = requests.Request('GET', features_url, params=params).prepare().url
            print(f"Pagina {page_num}: Verzoek naar de API:\n  {request_url}")
            
            response = requests.get(features_url, params=params, timeout=60)
            response.raise_for_status()
            
            data = response.json()
            features_this_page = data.get('features', [])
            
            if not features_this_page:
                print("Geen (verdere) features gevonden op deze pagina.")
                break
                
            all_features.extend(features_this_page)
            print(f"  - {len(features_this_page)} features ontvangen. Totaal nu: {len(all_features)}.")
            
            next_link = next((link['href'] for link in data.get('links', []) if link.get('rel') == 'next'), None)
            features_url = next_link
            params = {}  # Following requests just use URL of next_link.

        except requests.exceptions.RequestException as e:
            print(f"[-] Fout tijdens het verbinden met de server: {e}")
            return

    if all_features:
        feature_collection = {"type": "FeatureCollection", "features": all_features}
        output_filename = os.path.join(OUTPUT_DIR, f"{collection_id}_utrecht.geojson")
        
        try:
            with open(output_filename, 'w', encoding='utf-8') as f:
                json.dump(feature_collection, f, indent=2, ensure_ascii=False)
            print(f"[+] Succes: {len(all_features)} features opgeslagen in '{os.path.abspath(output_filename)}'")
        except IOError as e:
            print(f"[-] Fout bij het schrijven naar bestand: {e}")
    else:
        print(f"Geen features gevonden voor collectie '{collection_id}' met de opgegeven parameters.")



print("=====================================================================")
print("               BRT TOP10NL OGC API - GeoJSON Downloader              ")
print("=====================================================================")

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Directory '{OUTPUT_DIR}' aangemaakt.")

for collection in TARGET_COLLECTIONS:
    download_features_for_collection(collection)
    
print("\n--- Downloadproces voltooid. ---")

### Downloader

downloads ALL data from PDOK on a topic within NL

yes, incredibly inefficient, bbox not currently working in API calls.

In [ ]:
import requests
import json
import os

API_BASE_URL = 'https://api.pdok.nl/brt/top10nl/ogc/v1'
TARGET_COLLECTIONS = ['waterdeel_vlak']
LIMIT = 10000

OUTPUT_DIR_FULL_DATASET = 'geojson_data_nederland'

def download_full_collection(collection_id):
    """
    Downloadt ALLE features voor een collectie, zonder geografische filter.
    """
    print(f"\n--- Starten met downloaden van VOLLEDIGE collectie: {collection_id} ---")
    
    features_url = f"{API_BASE_URL}/collections/{collection_id}/items"
    
    params = {
        'f': 'json',
        'limit': LIMIT
    }
    
    all_features = []
    page_num = 1
    
    while features_url:
        try:
            print(f"Pagina {page_num}: Verzoek naar de API...")
            response = requests.get(features_url, params=params, timeout=90) # Langere timeout
            response.raise_for_status()
            
            data = response.json()
            features_this_page = data.get('features', [])
            
            if not features_this_page:
                print("Geen (verdere) features gevonden, download compleet.")
                break
                
            all_features.extend(features_this_page)
            print(f"  - {len(features_this_page)} features ontvangen. Totaal nu: {len(all_features)}.")
            
            next_link = next((link['href'] for link in data.get('links', []) if link.get('rel') == 'next'), None)
            features_url = next_link
            params = {}

        except requests.exceptions.RequestException as e:
            print(f"[-] Fout tijdens het verbinden: {e}")
            return

    if all_features:
        feature_collection = {"type": "FeatureCollection", "features": all_features}
        output_filename = os.path.join(OUTPUT_DIR_FULL_DATASET, f"{collection_id}_nederland.geojson")
        
        print(f"\n[+] Opslaan van {len(all_features)} features naar '{output_filename}'...")
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(feature_collection, f) # Geen indent voor kleiner bestand
        print(f"[+] Succesvol opgeslagen: {os.path.abspath(output_filename)}")
    else:
        print(f"Geen features gevonden voor collectie '{collection_id}'.")

print("=====================================================================")
print("         BRT TOP10NL OGC API - VOLLEDIGE DATASET DOWNLOADER        ")
print("=====================================================================")

if not os.path.exists(OUTPUT_DIR_FULL_DATASET):
    os.makedirs(OUTPUT_DIR_FULL_DATASET)

for collection in TARGET_COLLECTIONS:
    download_full_collection(collection)
    
print("\n--- Downloadproces voltooid. ---")

### Filtering

In [ ]:
import json
import os
from shapely.geometry import shape, box


SOURCE_DIR = 'geojson_data_nederland'
OUTPUT_DIR_FILTERED = 'geojson_data_utrecht'
TARGET_COLLECTIONS = ['waterdeel_vlak']
BBOX_COORDS = (4.75, 51.90, 5.57, 52.39)
utrecht_bbox = box(*BBOX_COORDS)

def filter_geojson_file(filename):
    """
    Leest een GeoJSON-bestand en filtert de features op basis van de Utrecht bbox.
    """
    source_path = os.path.join(SOURCE_DIR, filename)
    if not os.path.exists(source_path):
        print(f"[-] Bronbestand niet gevonden: {source_path}")
        return

    print(f"\n--- Filteren van bestand: {filename} ---")
    
    with open(source_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    original_features = data.get('features', [])
    print(f"Totaal {len(original_features)} features gevonden in bronbestand.")
    
    filtered_features = []
    for feature in original_features:
        if 'geometry' in feature and feature['geometry']:
            feature_shape = shape(feature['geometry'])
            
            if feature_shape.intersects(utrecht_bbox):
                filtered_features.append(feature)

    print(f"[+] {len(filtered_features)} features gevonden die de Utrecht bbox doorsnijden.")

    if filtered_features:
        feature_collection = {"type": "FeatureCollection", "features": filtered_features}
        output_filename = os.path.join(OUTPUT_DIR_FILTERED, filename.replace('_nederland', '_utrecht_filtered'))
        
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(feature_collection, f, indent=2)
        print(f"[+] Gefilterde data opgeslagen in: {os.path.abspath(output_filename)}")

if __name__ == "__main__":
    print("=====================================================================")
    print("              LOKALE GEOJSON BBOX FILTER (UTRECHT)                 ")
    print("=====================================================================")

    if not os.path.exists(OUTPUT_DIR_FILTERED):
        os.makedirs(OUTPUT_DIR_FILTERED)

    for collection_name in TARGET_COLLECTIONS:
        filter_geojson_file(f"{collection_name}_nederland.geojson")
        
    print("\n--- Filterproces voltooid. ---")

## LLM instructing polygon operations

### Create polygons

In [ ]:
import os
import json
import math
from dataclasses import dataclass
from typing import List, Dict, Any, Optional, Tuple

# Geospatial
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union
from pyproj import CRS, Transformer
from lxml import etree as ET

# Utrecht center (WGS84)
UTRECHT_LAT = 52.0907
UTRECHT_LON = 5.1214

# Coordinate reference systems
CRS_WGS84 = CRS.from_epsg(4326)
CRS_RDNEW = CRS.from_epsg(28992)  # Amersfoort / RD New, meters

TRANSFORMER_TO_RD = Transformer.from_crs(CRS_WGS84, CRS_RDNEW, always_xy=True)
TRANSFORMER_TO_WGS84 = Transformer.from_crs(CRS_RDNEW, CRS_WGS84, always_xy=True)

# Radius constraint (1 km)
MAX_RADIUS_M = 1000.0

# Output directory
OUTPUT_DIR = "test_polygons"

@dataclass
class ThemedPolygon:
    name: str
    theme: str
    polygon: Polygon
    properties: Dict[str, Any]

def wgs84_to_rd(lon: float, lat: float) -> Tuple[float, float]:
    return TRANSFORMER_TO_RD.transform(lon, lat)

def rd_to_wgs84(x: float, y: float) -> Tuple[float, float]:
    return TRANSFORMER_TO_WGS84.transform(x, y)

def generate_regular_polygon(center_xy: Tuple[float, float], radius_m: float, n: int, angle_deg: float = 0.0) -> Polygon:
    cx, cy = center_xy
    pts = []
    for i in range(n):
        theta = math.radians(angle_deg + (360.0 * i / n))
        x = cx + radius_m * math.cos(theta)
        y = cy + radius_m * math.sin(theta)
        pts.append((x, y))
    if pts[0] != pts[-1]:
        pts.append(pts[0])
    return Polygon(pts)

def ensure_within_radius(poly: Polygon, center_xy: Tuple[float, float], max_radius_m: float) -> Polygon:
    cx, cy = center_xy
    max_dist = 0.0
    for x, y in poly.exterior.coords:
        d = math.hypot(x - cx, y - cy)
        max_dist = max(max_dist, d)
    if max_dist <= max_radius_m or max_dist == 0:
        return poly
    scale = max_radius_m / max_dist
    scaled_coords = []
    for x, y in poly.exterior.coords:
        sx = cx + (x - cx) * scale
        sy = cy + (y - cy) * scale
        scaled_coords.append((sx, sy))
    return Polygon(scaled_coords)

def polygon_or_multi_to_poslist_wgs84(geom: Polygon | MultiPolygon) -> List[List[Tuple[float,float]]]:
    """
    Convert Polygon/MultiPolygon (RD) to a list of rings in WGS84:
    Returns list of rings (each ring is list of (lon,lat)), outer ring first, holes after.
    For MultiPolygon, rings are concatenated as separate outer rings followed by their holes.
    """
    rings: List[List[Tuple[float,float]]] = []
    def ring_to_lonlat(coords):
        return [rd_to_wgs84(x, y) for x, y in coords]
    if isinstance(geom, Polygon):
        rings.append(ring_to_lonlat(list(geom.exterior.coords)))
        for interior in geom.interiors:
            rings.append(ring_to_lonlat(list(interior.coords)))
    else:
        for poly in geom.geoms:
            rings.append(ring_to_lonlat(list(poly.exterior.coords)))
            for interior in poly.interiors:
                rings.append(ring_to_lonlat(list(interior.coords)))
    return rings

def polygon_to_gml(themed_poly: ThemedPolygon) -> bytes:
    """
    Serialize Polygon or MultiPolygon to GML 3.2 as a simple feature with:
    - name, theme, properties
    - geometry as gml:Polygon or gml:MultiSurface with gml:PolygonPatches (outer ring + holes)
    We prioritize compatibility: each outer ring is written as its own gml:Polygon within gml:MultiSurface when needed.
    Coordinates are lon lat in EPSG:4326.
    """
    ns_gml = "http://www.opengis.net/gml"
    ns_xsi = "http://www.w3.org/2001/XMLSchema-instance"
    NSMAP = {"gml": ns_gml, "xsi": ns_xsi}

    feature = ET.Element("FeatureCollection", nsmap=NSMAP)
    feature_member = ET.SubElement(feature, "featureMember")
    feat = ET.SubElement(feature_member, "ThemedPolygon")
    ET.SubElement(feat, "name").text = themed_poly.name
    ET.SubElement(feat, "theme").text = themed_poly.theme

    props_el = ET.SubElement(feat, "properties")
    for k, v in themed_poly.properties.items():
        prop = ET.SubElement(props_el, k)
        prop.text = json.dumps(v) if isinstance(v, (dict, list)) else str(v)

    geom = themed_poly.polygon
    # If geometry is MultiPolygon, write a gml:MultiSurface; else gml:Polygon
    def write_polygon(parent_el, poly: Polygon, poly_id: str):
        poly_el = ET.SubElement(parent_el, f"{{{ns_gml}}}Polygon", attrib={f"{{{ns_gml}}}id": poly_id, "srsName": "urn:ogc:def:crs:EPSG::4326"})
        # exterior
        exterior = ET.SubElement(poly_el, f"{{{ns_gml}}}exterior")
        lr = ET.SubElement(exterior, f"{{{ns_gml}}}LinearRing")
        poslist = ET.SubElement(lr, f"{{{ns_gml}}}posList")
        ext_lonlat = [rd_to_wgs84(x, y) for x, y in poly.exterior.coords]
        poslist.text = " ".join([f"{lon:.8f} {lat:.8f}" for lon, lat in ext_lonlat])
        # holes
        for interior in poly.interiors:
            interior_el = ET.SubElement(poly_el, f"{{{ns_gml}}}interior")
            lr_i = ET.SubElement(interior_el, f"{{{ns_gml}}}LinearRing")
            poslist_i = ET.SubElement(lr_i, f"{{{ns_gml}}}posList")
            int_lonlat = [rd_to_wgs84(x, y) for x, y in interior.coords]
            poslist_i.text = " ".join([f"{lon:.8f} {lat:.8f}" for lon, lat in int_lonlat])

    if isinstance(geom, Polygon):
        write_polygon(feat, geom, themed_poly.name)
    else:
        ms = ET.SubElement(feat, f"{{{ns_gml}}}MultiSurface", attrib={"srsName": "urn:ogc:def:crs:EPSG::4326"})
        for idx, poly in enumerate(geom.geoms):
            sm = ET.SubElement(ms, f"{{{ns_gml}}}surfaceMember")
            write_polygon(sm, poly, f"{themed_poly.name}_{idx}")

    return ET.tostring(feature, pretty_print=True, xml_declaration=True, encoding="UTF-8")

def make_polygon(name: str, theme: str, polygon: Polygon | MultiPolygon, properties: Optional[Dict[str, Any]] = None, out_dir: str = OUTPUT_DIR) -> str:
    """
    Create a polygon feature with metadata and write to a GML file. Returns the file path.
    """
    if properties is None:
        properties = {}
    os.makedirs(out_dir, exist_ok=True)
    tp = ThemedPolygon(name=name, theme=theme, polygon=polygon, properties=properties)
    gml_bytes = polygon_to_gml(tp)
    filepath = os.path.join(out_dir, f"{name}.gml")
    with open(filepath, "wb") as f:
        f.write(gml_bytes)
    return filepath

def cut_polygon(base_polygon, reference_polygon):
    """
    Remove overlapping area of reference_polygon from base_polygon.
    Returns Polygon or MultiPolygon.
    """
    if not base_polygon.is_valid:
        base_polygon = base_polygon.buffer(0)
    if not reference_polygon.is_valid:
        reference_polygon = reference_polygon.buffer(0)
    return base_polygon.difference(reference_polygon)

def load_gml_polygon(filepath: str) -> Tuple[Polygon | MultiPolygon, Dict[str, Any]]:
    """
    Load a GML file created by this script and return shapely geometry and properties including 'name' and 'theme'.
    Supports gml:Polygon and gml:MultiSurface outputs as written by polygon_to_gml.
    """
    ns_gml = "http://www.opengis.net/gml"
    tree = ET.parse(filepath)
    root = tree.getroot()

    # Basic props
    name_el = root.find(".//name")
    theme_el = root.find(".//theme")
    props = {
        "name": name_el.text if name_el is not None else "",
        "theme": theme_el.text if theme_el is not None else ""
    }

    # Try MultiSurface first
    ms = root.find(f".//{{{ns_gml}}}MultiSurface")
    polys: List[Polygon] = []
    if ms is not None:
        for surface_member in ms.findall(f".//{{{ns_gml}}}surfaceMember"):
            poly_el = surface_member.find(f".//{{{ns_gml}}}Polygon")
            if poly_el is None:
                continue
            # exterior
            posList_el = poly_el.find(f".//{{{ns_gml}}}exterior/{{{ns_gml}}}LinearRing/{{{ns_gml}}}posList")
            if posList_el is None or not posList_el.text:
                continue
            nums = [float(v) for v in posList_el.text.strip().split()]
            coords_lonlat = [(nums[i], nums[i+1]) for i in range(0, len(nums), 2)]
            coords_xy = [wgs84_to_rd(lon, lat) for lon, lat in coords_lonlat]
            # interiors
            interiors_xy = []
            for interior in poly_el.findall(f".//{{{ns_gml}}}interior"):
                pos_i = interior.find(f".//{{{ns_gml}}}LinearRing/{{{ns_gml}}}posList")
                if pos_i is not None and pos_i.text:
                    nums_i = [float(v) for v in pos_i.text.strip().split()]
                    coords_lonlat_i = [(nums_i[i], nums_i[i+1]) for i in range(0, len(nums_i), 2)]
                    interiors_xy.append([wgs84_to_rd(lon, lat) for lon, lat in coords_lonlat_i])
            poly = Polygon(coords_xy, holes=[ring for ring in interiors_xy if len(ring) >= 4])
            polys.append(poly)
    else:
        # Single Polygon
        poly_el = root.find(f".//{{{ns_gml}}}Polygon")
        if poly_el is None:
            raise ValueError(f"No gml:Polygon found in {filepath}")
        posList_el = poly_el.find(f".//{{{ns_gml}}}exterior/{{{ns_gml}}}LinearRing/{{{ns_gml}}}posList")
        if posList_el is None or not posList_el.text:
            raise ValueError(f"No coordinates found in {filepath}")
        nums = [float(v) for v in posList_el.text.strip().split()]
        coords_lonlat = [(nums[i], nums[i+1]) for i in range(0, len(nums), 2)]
        coords_xy = [wgs84_to_rd(lon, lat) for lon, lat in coords_lonlat]
        interiors_xy = []
        for interior in poly_el.findall(f".//{{{ns_gml}}}interior"):
            pos_i = interior.find(f".//{{{ns_gml}}}LinearRing/{{{ns_gml}}}posList")
            if pos_i is not None and pos_i.text:
                nums_i = [float(v) for v in pos_i.text.strip().split()]
                coords_lonlat_i = [(nums_i[i], nums_i[i+1]) for i in range(0, len(nums_i), 2)]
                interiors_xy.append([wgs84_to_rd(lon, lat) for lon, lat in coords_lonlat_i])
        poly = Polygon(coords_xy, holes=[ring for ring in interiors_xy if len(ring) >= 4])
        polys = [poly]

    geom = polys[0] if len(polys) == 1 else MultiPolygon(polys)
    return geom, props

def generate_sample_polygons() -> List[ThemedPolygon]:
    """
    Create several overlapping polygons around Utrecht center, each with ≥8 vertices, within ~1 km radius.
    Themes: forest, factory, pond, park, residential, commercial, plus encompassing 'city'.
    """
    cx, cy = wgs84_to_rd(UTRECHT_LON, UTRECHT_LAT)

    specs = [
        ("forest_zone", "forest", 120, 80, 220, 8, 10),
        ("factory_zone", "factory", -150, 60, 240, 9, 0),
        ("pond_zone", "pond", 60, -130, 180, 10, 20),
        ("park_zone", "park", -60, -100, 200, 8, 15),
        ("residential_zone", "residential", 200, -40, 230, 8, 5),
        ("commercial_zone", "commercial", -220, -30, 210, 9, -10),
    ]
    themed_polys: List[ThemedPolygon] = []

    for name, theme, dx, dy, radius, sides, rot in specs:
        center = (cx + dx, cy + dy)
        poly = generate_regular_polygon(center, radius, sides, rot)
        poly = ensure_within_radius(poly, (cx, cy), MAX_RADIUS_M)
        props = {
            "description": f"{theme.capitalize()} area near Utrecht center",
            "priority": {"forest": 2, "pond": 3, "park": 2, "factory": 1, "residential": 2, "commercial": 1}.get(theme, 1),
        }
        themed_polys.append(ThemedPolygon(name=name, theme=theme, polygon=poly, properties=props))

    # City polygon that encompasses all others (buffered union)
    union_poly = unary_union([tp.polygon for tp in themed_polys]).buffer(80)
    union_poly = ensure_within_radius(union_poly, (cx, cy), MAX_RADIUS_M)
    themed_polys.append(ThemedPolygon(
        name="city_boundary",
        theme="city",
        polygon=union_poly,
        properties={"description": "City boundary encompassing prototypes", "priority": 0}
    ))
    return themed_polys

def write_sample_gmls(polys: List[ThemedPolygon], out_dir: str = OUTPUT_DIR) -> List[str]:
    paths = []
    for tp in polys:
        paths.append(make_polygon(tp.name, tp.theme, tp.polygon, tp.properties, out_dir))
    return paths

print("Cell 1 loaded. Functions available: make_polygon(), cut_polygon(), load_gml_polygon(), generate_sample_polygons().")

In [ ]:
# Cell 2: Generate polygons, save to GML, and demo a cut operation

# themed_polys = generate_sample_polygons()
# paths = write_sample_gmls(themed_polys)
# print(f"Wrote {len(paths)} GML files to {OUTPUT_DIR}:")
# for p in sorted(paths):
    # print(" -", os.path.basename(p))

def cut_by_theme(base_name: str, themes_to_cut: List[str], out_dir: str = OUTPUT_DIR) -> str:
    files = [os.path.join(out_dir, f) for f in os.listdir(out_dir) if f.endswith(".gml")]
    base_poly = None
    base_props = None
    refs = []
    for f in files:
        poly, props = load_gml_polygon(f)
        if props.get("name") == base_name:
            base_poly = poly
            base_props = props
        else:
            if props.get("theme") in themes_to_cut:
                refs.append(poly)
    if base_poly is None:
        raise ValueError(f"Base polygon {base_name} not found.")
    result = base_poly if not refs else cut_polygon(base_poly, unary_union(refs))
    out_name = f"{base_name}_cut_{'_'.join(themes_to_cut)}"
    out_path = make_polygon(out_name, base_props.get("theme", "unknown"), result,
                            {"cut_from": base_name, "removed_themes": ",".join(themes_to_cut)})
    return out_path

# result_path = cut_by_theme("park_zone", ["factory", "pond"])
# print(f"Example cut result written to: {result_path}")

In [ ]:
# Cell 3: GPT plan call and local execution

import os
import json
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

def call_gpt_with_cutting(instruction: str,
                          available_polygons: List[Dict[str, Any]],
                          allowed_themes: List[str],
                          model: str = "gpt-4.1") -> Dict[str, Any]:
    """
    Ask GPT for a JSON plan indicating which base polygon to modify and which themes to cut.
    We then parse the JSON and execute locally with cut_polygon().
    """
    
    api_key = os.environ.get("OPENAI_API_KEY")
    client = OpenAI(api_key=api_key)

    system_prompt = (
        "You are a planning assistant. Propose a cutting plan to remove overlaps from one base polygon "
        "using themed polygons. Only choose themes from the provided list. "
        "Return pure JSON with keys: base_name (string), remove_themes (array of strings), notes (string). "
        "No extra text."
    )

    user_context = {
        "available_polygons": available_polygons,
        "allowed_themes": allowed_themes,
        "instruction": instruction
    }

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": json.dumps(user_context)}
        ],
        temperature=0.2,
    )

    raw_text = resp.choices[0].message.content.strip()
    plan_json = None
    try:
        plan_json = json.loads(raw_text)
    except Exception:
        # Basic recovery if extra text sneaks in
        start = raw_text.find("{")
        end = raw_text.rfind("}")
        if start != -1 and end != -1 and end > start:
            try:
                plan_json = json.loads(raw_text[start:end+1])
            except Exception:
                pass

    return {"plan_json": plan_json, "raw_response": raw_text}

def execute_gpt_cut_plan(plan: Dict[str, Any], out_dir: str = OUTPUT_DIR) -> Optional[str]:
    """
    Execute a plan like:
      { "base_name": "residential_zone", "remove_themes": ["factory","commercial"], "notes": "..." }
    Returns path to the new GML result, or None on failure.
    """
    if not plan or "base_name" not in plan or "remove_themes" not in plan:
        return None
    base_name = plan["base_name"]
    themes = [t for t in plan["remove_themes"] if isinstance(t, str)]
    return cut_by_theme(base_name, themes, out_dir)

# Prepare context for GPT
available = [{"name": os.path.splitext(f)[0], "theme": load_gml_polygon(os.path.join(OUTPUT_DIR, f))[1]["theme"]}
             for f in os.listdir(OUTPUT_DIR) if f.endswith(".gml")]
allowed = sorted(list({a["theme"] for a in available if a["theme"] != "city"}))

print("Available polygons:", available)
print("Allowed themes:", allowed)

if os.environ.get("OPENAI_API_KEY"):
    instruction = "Remove any polygons connected or related to nature from the city polygon."
    out = call_gpt_with_cutting(instruction, available, allowed, model="gpt-4.1")
    print("Raw model response:", out["raw_response"])
    if out["plan_json"]:
        res_path = execute_gpt_cut_plan(out["plan_json"], OUTPUT_DIR)
        if res_path:
            print(f"Executed plan; result saved to: {res_path}")
        else:
            print("Plan could not be executed.")
    else:
        print("Could not parse plan JSON from model.")
else:
    print("OpenAI not available or API key not set; skipping GPT demo.")

### Reading in .gml polygons

In [1]:
import geopandas as gpd
from lxml import etree as ET
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union
from pyproj import Transformer
import os
import matplotlib.pyplot as plt

In [2]:
def extract_gml_polygons(gml_path, transform_to_epsg: int | None = None):
    """
    Return list of dicts: { 'ID', 'statcode', 'jrstatcode', 'statnaam', 'rubriek', 'geometry' }
    geometry is a shapely Polygon or MultiPolygon (or None if no geometry).
    If transform_to_epsg is given (e.g. 4326), coordinates are reprojected to that CRS.
    """
    ns = {'ogr': "http://ogr.maptools.org/", 'gml': "http://www.opengis.net/gml/3.2"}
    tree = ET.parse(gml_path)
    root = tree.getroot()

    transformer = None
    if transform_to_epsg is not None:
        transformer = Transformer.from_crs(28992, transform_to_epsg, always_xy=True)

    def maybe_transform(x, y):
        return transformer.transform(x, y) if transformer else (x, y)

    out = []
    for feat in root.findall('.//ogr:buurt_boundaries', ns):
        gid = feat.get('{http://www.opengis.net/gml/3.2}id') or feat.get('gml:id')
        statcode = (feat.find('ogr:statcode', ns).text if feat.find('ogr:statcode', ns) is not None else None)
        jrstatcode = (feat.find('ogr:jrstatcode', ns).text if feat.find('ogr:jrstatcode', ns) is not None else None)
        statnaam = (feat.find('ogr:statnaam', ns).text if feat.find('ogr:statnaam', ns) is not None else None)
        rubriek = (feat.find('ogr:rubriek', ns).text if feat.find('ogr:rubriek', ns) is not None else None)

        # collect all gml:posList under this feature (one per polygon patch)
        poslists = feat.findall('.//gml:posList', ns)
        polygons = []
        for p in poslists:
            txt = (p.text or "").strip()
            if not txt:
                continue
            nums = [float(v) for v in txt.split()]
            coords = [(nums[i], nums[i+1]) for i in range(0, len(nums), 2)]
            coords = [maybe_transform(x, y) for x, y in coords]
            try:
                poly = Polygon(coords)
                if not poly.is_valid:
                    poly = poly.buffer(0)
                polygons.append(poly)
            except Exception:
                continue

        if not polygons:
            geometry = None
        elif len(polygons) == 1:
            geometry = polygons[0]
        else:
            geometry = MultiPolygon(polygons)

        out.append({
            "ID": gid,
            "statcode": statcode,
            "jrstatcode": jrstatcode,
            "statnaam": statnaam,
            "rubriek": rubriek,
            "geometry": geometry
        })
    return out

In [ ]:
output_dir = "all_boundaries_processed"
gml_file = "/home/fkok2/all_boundaries/buurt_boundaries.gml"
output_basename = os.path.basename(gml_file).replace(".gml", "")

# features = extract_gml_polygons(gml_file, transform_to_epsg=4326)
features = extract_gml_polygons(gml_file) # RD coordinates

gdf = gpd.GeoDataFrame(features, geometry='geometry', crs="EPSG:4326")
# gdf = gdf.set_crs("EPSG:28992", inplace=False) # Set to accurate Dutch coordinate system, TODO: fix this, currently does not work
gdf['full_file_path'] = gml_file
gdf['filename'] = os.path.basename(gml_file)

print("Features parsed:", len(gdf))
print(gdf[['ID','statcode','statnaam','rubriek','filename', 'full_file_path']].head())

os.makedirs(output_dir, exist_ok=True)
out_gpkg = f"{output_basename}.gpkg"
out_geojson = f"{output_basename}.geojson"
out_pickle = f"{output_basename}_gdf.pkl"

gdf.to_file(output_dir + "/" + out_gpkg, driver="GPKG")
print("Saved GeoPackage:", out_gpkg)
gdf.to_file(output_dir + "/" + out_geojson, driver="GeoJSON")
print("Saved GeoJSON:", out_geojson)
gdf.to_pickle(output_dir + "/" + out_pickle)
print("Saved GeoDataFrame (pickle):", out_pickle)

Features parsed: 1000
                   ID    statcode                   statnaam rubriek  \
0  buurt_boundaries.0  BU00340101        Centrum Haven Noord   buurt   
1  buurt_boundaries.1  BU00340102         Centrum Haven Zuid   buurt   
2  buurt_boundaries.2  BU00340201  Rozenwerf en Tuinderswerf   buurt   
3  buurt_boundaries.3  BU00340202                 Achterwerf   buurt   
4  buurt_boundaries.4  BU00340203     Goedewerf en Wittewerf   buurt   

               filename                                   full_file_path  
0  buurt_boundaries.gml  /home/fkok2/all_boundaries/buurt_boundaries.gml  
1  buurt_boundaries.gml  /home/fkok2/all_boundaries/buurt_boundaries.gml  
2  buurt_boundaries.gml  /home/fkok2/all_boundaries/buurt_boundaries.gml  
3  buurt_boundaries.gml  /home/fkok2/all_boundaries/buurt_boundaries.gml  
4  buurt_boundaries.gml  /home/fkok2/all_boundaries/buurt_boundaries.gml  
Saved GeoPackage: buurt_boundaries.gpkg
Saved GeoJSON: buurt_boundaries.geojson
Saved GeoDataFr

### Inspect PDOS geopackage

Downloaded from NGR directly, the Top10NL package.

In [4]:
import sqlite3
import geopandas as gpd

In [ ]:
# running a query

file_path = 'data/spatial_genai_storage/data_PDOK/top10nl_Compleet.gpkg'
sql_tables = "SELECT name, type FROM sqlite_master WHERE type IN ('table','view') ORDER BY name;"
sql_contents = "SELECT * FROM gpkg_contents;"
sql_columns = "SELECT * FROM gpkg_geometry_columns;"
sql_spatialref = "SELECT * FROM gpkg_spatial_ref_sys;"

conn = sqlite3.connect(file_path)
cur = conn.cursor()

cur.execute(sql_tables)
tables = cur.fetchall()

cur.execute(sql_contents)
contents = cur.fetchall()

cur.execute(sql_columns)
columns = cur.fetchall()

cur.execute(sql_spatialref)
spatialref = cur.fetchall()

conn.close()

print("Tables and Views:")
for t in tables:
    print(" -", t)

print("Contents:")
for c in contents:
    print(" -", c)

print("Columns:")
for col in columns:
    print(" -", col)

print("Spatial Reference:")
for s in spatialref:
    print(" -", s)

Tables and Views:
 - ('gpkg_contents', 'table')
 - ('gpkg_extensions', 'table')
 - ('gpkg_geometry_columns', 'table')
 - ('gpkg_spatial_ref_sys', 'table')
 - ('gpkg_tile_matrix', 'table')
 - ('gpkg_tile_matrix_set', 'table')
 - ('rtree_top10nl_functioneel_gebied_multivlak_geom', 'table')
 - ('rtree_top10nl_functioneel_gebied_multivlak_geom_node', 'table')
 - ('rtree_top10nl_functioneel_gebied_multivlak_geom_parent', 'table')
 - ('rtree_top10nl_functioneel_gebied_multivlak_geom_rowid', 'table')
 - ('rtree_top10nl_functioneel_gebied_punt_geom', 'table')
 - ('rtree_top10nl_functioneel_gebied_punt_geom_node', 'table')
 - ('rtree_top10nl_functioneel_gebied_punt_geom_parent', 'table')
 - ('rtree_top10nl_functioneel_gebied_punt_geom_rowid', 'table')
 - ('rtree_top10nl_functioneel_gebied_vlak_geom', 'table')
 - ('rtree_top10nl_functioneel_gebied_vlak_geom_node', 'table')
 - ('rtree_top10nl_functioneel_gebied_vlak_geom_parent', 'table')
 - ('rtree_top10nl_functioneel_gebied_vlak_geom_rowid', 't

SQLite queries can view more 'admin' like tables, geopandas can't, hence why SQL can be convenient/pleasant for browsing through the content of the database.

In [10]:
# for a given table, print the first 10 rows as a geodataframe
table = "rtree_top10nl_waterdeel_punt_geom"
sql = f"SELECT * FROM {table} LIMIT 10;"


conn = sqlite3.connect(file_path)
cur = conn.cursor()

cur.execute(sql)

rows = cur.fetchall()

conn.close()

df = gpd.GeoDataFrame(rows, columns=[desc[0] for desc in cur.description])
print(df)
print(df.columns)

    id           minx           maxx          miny          maxy
0   49  184042.281250  184042.312500  308618.18750  308618.21875
1   60  184239.000000  184239.031250  308836.12500  308836.15625
2   48  184278.953125  184278.984375  308634.81250  308634.84375
3  240  184648.093750  184648.125000  309124.12500  309124.15625
4   54  191361.765625  191361.796875  307237.03125  307237.06250
5   31  191391.328125  191391.359375  307249.62500  307249.68750
6   32  191461.031250  191461.062500  307305.21875  307305.25000
7   53  191474.656250  191474.687500  307320.96875  307321.03125
8   70  191477.531250  191477.562500  307360.21875  307360.28125
9   56  191504.890625  191504.921875  307367.25000  307367.28125
Index(['id', 'minx', 'maxx', 'miny', 'maxy'], dtype='object')


geopandas is easier for utilising bounding boxes as filter on what data to read. 

In [8]:
# querying on only data within Province Utrecht
table = "top10nl_waterdeel_vlak"

# boundary box province of utrecht in EPSG:28992 (Amersfoort / RD New)
bbox = (128000, 445000, 160000, 475000)  # min_x, min_y, max_x, max_y

gdf = gpd.read_file(file_path, layer=table, bbox=bbox)
print(f"Read {len(gdf)} features from layer '{table}' (bbox applied).")
print(gdf.head(3))

Read 11255 features from layer 'top10nl_waterdeel_vlak' (bbox applied).
    namespace   lokaalid   brontype bronactualiteit  \
0  NL.TOP10NL  110974666  luchtfoto      2013-01-01   
1  NL.TOP10NL  110985511  luchtfoto      2011-01-01   
2  NL.TOP10NL  118091232  luchtfoto      2013-01-01   

                                    bronbeschrijving  bronnauwkeurigheid  \
0  Een orthogerectificeerde fotografische opname ...                 0.1   
1  Een orthogerectificeerde fotografische opname ...                 0.1   
2  Een orthogerectificeerde fotografische opname ...                 0.1   

  mutatietype  typewater breedteklasse hoofdafwatering  ... naamnl naamfries  \
0        None  waterloop  6 - 12 meter             nee  ...   None      None   
1        None  waterloop  6 - 12 meter             nee  ...   None      None   
2        None  waterloop  6 - 12 meter              ja  ...   None      None   

   isbagnaam sluisnaam brugnaam objectbegintijd tijdstipregistratie  \
0       No

In [ ]:
import geopandas as gpd

file_path = 'data/spatial_genai_storage/data_PDOK/top10nl_Compleet.gpkg'
# manually selected tables with 'vlak' or 'multivlak' (POLYGON/MULTIPOLYGON)
# largely for simplicity for demo, in the end lines/points should be handled too
tables = [
    'top10nl_functioneel_gebied_multivlak',
    'top10nl_functioneel_gebied_vlak',
    'top10nl_gebouw_vlak',
    'top10nl_geografisch_gebied_multivlak',
    'top10nl_geografisch_gebied_vlak',
    'top10nl_plaats_multivlak',
    'top10nl_plaats_vlak',
    'top10nl_plantopografie_vlak',
    'top10nl_registratief_gebied_multivlak',
    'top10nl_registratief_gebied_vlak',
    'top10nl_terrein_vlak',
    'top10nl_waterdeel_vlak',
    'top10nl_wegdeel_vlak'
]

# make a text file to fill
output_file = "data/spatial_genai_storage/data_PDOK/llm_reference.txt"
bbox = (128000, 445000, 160000, 475000)  # min_x, min_y, max_x, max_y | Province Utrecht

for table in tables:
    gdf = gpd.read_file(file_path, layer=table, bbox=bbox)
    gdf = gdf.drop(columns=['geometry', 'namespace', 'brontype', 'bronactualiteit', 'bronbeschrijving', 'bronnauwkeurigheid'])
    with open(output_file, "a") as f:
        f.write(f"Table: {table}\n")
        for col in gdf.columns:
            unique_vals = gdf[col].unique()
            if (len(unique_vals) < 30) & (gdf[col].notnull().sum() > 0):
                f.write(f"- Column '{col}' has {len(unique_vals)} unique values: {unique_vals}\n")
        f.write("\n\n")
    print(f"Wrote reference for table '{table}' to {output_file}")  

Wrote reference for table 'top10nl_functioneel_gebied_multivlak' to data/spatial_genai_storage/PDOK/llm_reference.txt
Wrote reference for table 'top10nl_functioneel_gebied_vlak' to data/spatial_genai_storage/PDOK/llm_reference.txt
Wrote reference for table 'top10nl_gebouw_vlak' to data/spatial_genai_storage/PDOK/llm_reference.txt
Wrote reference for table 'top10nl_geografisch_gebied_multivlak' to data/spatial_genai_storage/PDOK/llm_reference.txt
Wrote reference for table 'top10nl_geografisch_gebied_vlak' to data/spatial_genai_storage/PDOK/llm_reference.txt
Wrote reference for table 'top10nl_plaats_multivlak' to data/spatial_genai_storage/PDOK/llm_reference.txt
Wrote reference for table 'top10nl_plaats_vlak' to data/spatial_genai_storage/PDOK/llm_reference.txt
Wrote reference for table 'top10nl_plantopografie_vlak' to data/spatial_genai_storage/PDOK/llm_reference.txt
Wrote reference for table 'top10nl_registratief_gebied_multivlak' to data/spatial_genai_storage/PDOK/llm_reference.txt
Wr

### Handling TIFF files